Run each cell in order to create a dataset of 180000 inputs and outputs and save to drive

In [9]:
import numpy as np
from numpy import random
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
key_names = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B','Cm','C#m','Dm','D#m','Em','Fm','F#m','Gm','G#m','Am','A#m','Bm']
N = 4

def next_key(chords):
  next = [[[0 for i in range(3)] for j in range(6)] for k in range(6)]
  for i in range(6):
    for j in range(6):
      next[i][j] = chords[i][j].copy()
      next[i][j][2] = next[i][j][2] + 1
    if next[i][4][2] == 12:
      next[i].pop()
      next[i].insert(0,next[i][4].copy())
      next[i][0][2] = next[i][0][2] - 12
  return next

def chord2token(chord):
  return chord[0]*5*15 + chord[1]*15 + chord[2] + 1

def chord2name(chord):
  estring = ['E','F','F#','G','G#','A','A#','B','C','C#','D','D#','E','F','F#','G','A','A#']
  astring = ['A','A#','B','C','C#','D','D#','E','F','F#','G','G#','A','A#','B','C','C#','D']
  dstring = ['D','D#','E','F','F#','G','G#','A','A#','B','C','C#','D','D#','E','F','F#','G']
  name = ''
  color = chord[0]
  shape = chord[1]
  bar = chord[2]
  if shape == 0: name = name + astring[bar+3]
  elif shape == 1: name = name + astring[bar]
  elif shape == 2: name = name + estring[bar+3]
  elif shape == 3: name = name + estring[bar]
  elif shape == 4: name = name + dstring[bar]
  if color == 1: name = name + 'm'
  return name

def token2chord(token):
  token = token - 1
  t1 = int(token/(5*15))
  t2 = int((token-(t1*5*15))/15)
  t3 = token - t1*5*15 - t2*15
  return [t1,t2,t3]

def report(index,inputs,outputs):
  for i in range(N):
    print(chord2name(token2chord(inputs[index][i])))
  print('key is ' + key_names[outputs[index]])

def make_seq2key_dataset():
  # specify csf chord representations
  c = [[[0,0,0],[0,1,3],[0,2,5],[0,3,8],[0,4,10],[0,0,12]], # 1
      [[1,4,0],[1,0,2],[1,1,5],[1,2,7],[1,3,10],[1,4,12]], # 2
      [[1,3,0],[1,4,2],[1,0,4],[1,1,7],[1,2,9],[1,3,12]], # 3
      [[0,3,1],[0,4,3],[0,0,5],[0,1,8],[0,2,10],[0,3,13]], # 4
      [[0,2,0],[0,3,3],[0,4,5],[0,0,7],[0,1,10],[0,2,12]], # 5
      [[1,1,0],[1,2,2],[1,3,5],[1,4,7],[1,0,9],[1,1,12]]] # 6
  csharp = next_key(c)
  d = next_key(csharp)
  dsharp = next_key(d)
  e = next_key(dsharp)
  f = next_key(e)
  fsharp = next_key(f)
  g = next_key(fsharp)
  gsharp = next_key(g)
  a = next_key(gsharp)
  asharp = next_key(a)
  b = next_key(asharp)
  keys = [c,csharp,d,dsharp,e,f,fsharp,g,gsharp,a,asharp,b]

  # tokenize inputs
  for i in range(len(keys)):
    for j in range(6):
      for k in range(6):
        keys[i][j][k] = chord2token(keys[i][j][k].copy())

  # specify progressions
  major_progs = [[1,1,4,3],[1,3,4,6],[1,3,6,4],[1,4,2,5],[1,4,5,4],[1,4,5,5],[1,4,6,5],[1,5,1,4],[1,5,4,6],
                    [1,5,6,2],[1,5,6,3,4,1,4,5],[1,5,6,3,4],[1,5,6,4],[1,5,6,5],[1,6,1,4],[1,6,2,4],[1,6,2,5],[1,6,4,3],
                    [1,6,4,5],[2,4,5,5],[2,4,6,5],[2,5,1,1],[2,5,1,4],[3,6,4,1],[4,1,2,6],[4,1,3,4],[4,1,5,6],[4,4,1,5],
                    [4,6,1,5],[4,6,3,1],[4,6,4,6],[5,1,6,5],[5,4,6,1],[5,6,4,1],[6,2,5,1],[6,4,1,5],[6,5,4,5,2,5,1,1],[6,5,4,5]]
  minor_progs = [[6,1,2,4],[6,1,5,4],[6,2,1,4],[6,2,3,2],[6,2,3,3],[6,2,4,3],[6,2,5,6],
                          [6,3,2,5],[6,4,1,5],[6,4,2,1],[6,4,2,3],[6,4,5,3],[6,4,5,5],[6,5,6,3,1,5,6,3,6],[6,5,6,3],
                          [6,5,3,4],[6,5,4,1,2,4,5,6],[6,5,4,1],[6,5,4,5],[2,6,3,4],[2,1,5,6],
                          [2,3,4,5],[2,4,3,5],[2,4,5,6],[3,6,2,5],[3,2,6,6],[3,4,1,6],[3,4,3,6],[4,6,3,1],[4,6,3,3],[4,1,6,3],
                          [4,2,6,3],[4,4,6,5],[4,5,6,1],[4,5,3,1],[5,2,3,6],[5,2,5,6]]

  # organize inputs and outputs
  # input: sequence of N tokens in a given key
  # output: one of 12 possible key labels
  prog_iterations = 200
  major_dataset = [[],[]]
  minor_dataset = [[],[]]
  for k in range(len(keys)):
    key = k
    for prog in range(len(major_progs)):
      for iteration in range(prog_iterations):
        sequence = [0]*N
        for n in range(N):
          voicing_index = random.randint(6)
          sequence[n] = keys[key][major_progs[prog][n%len(major_progs[prog])]-1][voicing_index]
        major_dataset[0].append(sequence)
        major_dataset[1].append(key)
    for prog in range(len(minor_progs)):
      for iteration in range(prog_iterations):
        sequence = [0]*N
        for n in range(N):
          voicing_index = random.randint(6)
          sequence[n] = keys[key][minor_progs[prog][n%len(minor_progs[prog])]-1][voicing_index]
        minor_dataset[0].append(sequence)
        minor_dataset[1].append(key)
  dataset = [major_dataset,minor_dataset]

  # print length of dataset
  num_sequences = len(dataset[0][0]) + len(dataset[1][0])
  print('seq2key dataset size: ' + str(num_sequences))

  # randomly "mute" (mask) some notes in the input sequence so the model learns how to fill in the blanks. This is directly inspired by the BERT model
  inputs = []
  outputs = []
  for i in range(num_sequences):
    color = random.randint(2)
    if len(dataset[color][0]) == 0:
      color = (color+1)%2
    rand_sample_index = random.randint(len(dataset[color][0]))
    #input_seq = mask(dataset[color][0][rand_sample_index]) # masking not currently implemented
    input_seq = dataset[color][0][rand_sample_index].copy()
    #output_seq = dataset[color][0][rand_sample_index] # masking not currently implemented
    output_key = dataset[color][1][rand_sample_index]
    inputs.append(input_seq)
    outputs.append(output_key)
    dataset[color][0].pop(rand_sample_index)
    dataset[color][1].pop(rand_sample_index)

  # convert inputs and outputs to tensors
  inputs = tf.convert_to_tensor(inputs, dtype=tf.int32)
  outputs = tf.convert_to_tensor(outputs, dtype=tf.int32)
  print()
  print('Shape of the input tensors: ' + str(tf.shape(inputs)))
  print('Shape of the output tensors: ' + str(tf.shape(outputs)))
  print()

  # show example
  idx = 8
  print('Input example: ' + str(inputs[idx].numpy()))
  print('Resulting output: ' + str(outputs[idx].numpy()))
  print()
  print('Example corresponds to:')
  report(idx,inputs,outputs)

  return inputs, outputs

In [8]:
inputs, outputs = make_seq2key_dataset()

seq2key dataset size: 180000

Shape of the input tensors: tf.Tensor([180000      4], shape=(2,), dtype=int32)
Shape of the output tensors: tf.Tensor([180000], shape=(1,), dtype=int32)

Input example: [105 112  38  46]
Resulting output: 9

Example corresponds to:
Bm
C#m
D
E
key is A


In [14]:
s2k_dataset_inputs = np.array(inputs)
np.save('/content/drive/My Drive/FretboardAI/s2k_dataset_inputs',s2k_dataset_inputs)

In [15]:
s2k_dataset_outputs = np.array(outputs)
np.save('/content/drive/My Drive/FretboardAI/s2k_dataset_outputs',s2k_dataset_outputs)